In [60]:
import spacy
import os
from functools import partial
from train_utils.metrics import *

from preprocessing.cleaning_utils import *

In [2]:
from datasets import load_dataset
from datasets import Value, ClassLabel, Features, DatasetDict
from datasets import load_metric


In [7]:
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, BertTokenizer

In [22]:
trained = "/home/vs428/Documents/n2c2_2022/outputs/2022-06-28/12-20-30/test_trainer/checkpoint-9500"

assessment_model_f = "/home/vs428/Documents/prodigy/assessment_model/model-best"
plan_model_f = "/home/vs428/Documents/prodigy/plan_subsection_model/model-best"

In [10]:
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")
model = BertForSequenceClassification.from_pretrained(trained, num_labels=4)


In [91]:

# Read MIMIC notes
# notes = pd.read_csv(cfg.data.mimic_data_dir + "NOTEEVENTS.csv")

# create hf Dataset
classes = ['Not Relevant', 'Neither', 'Indirect', 'Direct']

# instead we will use the raw text for now
features = Features({
    'ROW ID':Value("int64"),
    'HADM ID':Value("int64"),
    'Assessment':Value("string"),
    'Plan Subsection':Value("string"),
    "Relation":Value("string")
}) 

dataset = load_dataset("csv", data_files={
                            "train":"/home/vs428/project/n2c2/2022/Data/train.csv",
                            "valid":"/home/vs428/project/n2c2/2022/Data/dev.csv",
                        },
                       features=features)


Using custom data configuration default-b1948d86214b7517
Reusing dataset csv (/home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/2 [00:00<?, ?it/s]

In [92]:
dataset['train'] = dataset['train'].shard(num_shards=1000, index=0)
dataset['valid'] = dataset['valid'].shard(num_shards=100, index=0)


In [93]:
dataset = dataset.class_encode_column("Relation")
dataset = dataset.rename_column("Relation", "label")


Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-b43d1659b503db36.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-1c80317fa3b1799d.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-cca2bc4cceeb4c43.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-711f7a4ba5235efb.arrow
Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cach

In [75]:
nlp_assessment = spacy.load(assessment_model_f, exclude="parser")
nlp_plan = spacy.load(plan_model_f, exclude="parser")


loading configuration file /tmp/tmp2uakt6uw/config.json
Model config RobertaConfig {
  "_name_or_path": "/tmp/tmp2uakt6uw/config.json",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

Didn't find file /tmp/tmp2uakt6uw/added_tokens.json. We won't load it.
loading file /tmp/tmp2uakt6uw/vocab.json
loading file /tmp/tmp2uakt6uw/merges.txt
loading file /tmp/tmp2uakt6uw/tokenizer.json
loading file None
loading file /tmp/tmp2uakt6uw/special_to

In [94]:
# add the named entities
dataset['train'] = dataset['train'].map(partial(add_ner_assessment, nlp=nlp_assessment))
dataset['train'] = dataset['train'].map(partial(add_ner_plan, nlp=nlp_plan))        

dataset['valid'] = dataset['valid'].map(partial(add_ner_assessment, nlp=nlp_assessment))
dataset['valid'] = dataset['valid'].map(partial(add_ner_plan, nlp=nlp_plan))        


Loading cached processed dataset at /home/vs428/.cache/huggingface/datasets/csv/default-b1948d86214b7517/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58/cache-3eb13b9046685257.arrow


  0%|          | 0/5 [00:00<?, ?ex/s]

/gpfs/ysm/project/rtaylor/vs428/conda_envs/n2c2_env/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


  0%|          | 0/6 [00:00<?, ?ex/s]

  0%|          | 0/6 [00:00<?, ?ex/s]

In [95]:
# we ASSUME that the ner labels we want are lowercase, UNLIKE the standard ones in the model
spans = [x for x in nlp_plan.get_pipe("ner").labels if x.islower()] + [x for x in nlp_assessment.get_pipe("ner").labels if x.islower()]

tokens = []
for span in spans:
    tokens.append("<" + span + ">")
    tokens.append("</" + span + ">")            

# add the span tags to the vocab
_ = tokenizer.add_tokens(tokens)


In [105]:
[x for x in nlp_plan.get_pipe("ner").labels if x.islower()]

['complication_related_to_problem',
 'event_related_to_problem',
 'organ_failure_related_to_problem',
 'problem']

In [96]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding


In [97]:
# arguments for Trainer
test_args = TrainingArguments(
    output_dir = "test_output",
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 4,   
    dataloader_drop_last = False    
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [98]:
# metrics to track
acc = load_metric("accuracy")
macrof1 = load_metric("f1")    

# create metric_dict for compute_metrics
metric_dict = {}
metric_dict['accuracy'] = {"metric":acc}
metric_dict['f1-macro'] = {"metric":macrof1, "average":"macro"}

# tokenize
dataset = dataset.map(partial(tokenize_function, tokenizer=tokenizer), batched=True)

print(tokenizer.decode(dataset['valid'][0]['input_ids']))
print(tokenizer.decode(dataset['valid'][1]['input_ids']))
print(tokenizer.decode(dataset['valid'][2]['input_ids']))
print(tokenizer.decode(dataset['valid'][3]['input_ids']))
print(tokenizer.decode(dataset['valid'][4]['input_ids']))
print(tokenizer.decode(dataset['valid'][5]['input_ids']))


# cast as pytorch tensors and select a subset of columns we want
dataset['train'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset['valid'].set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])    

# create collator
data_collator = DataCollatorWithPadding(tokenizer,
                                        max_length=512, 
                                        padding="max_length",
                                        return_tensors="pt")    
# create Trainer
trainer = Trainer(
    model=model,
    args=test_args,
    compute_metrics=partial(compute_metrics, metric_dict=metric_dict),
    data_collator=data_collator,
)    


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

[CLS] a 60 year old woman with <secondary_problem> recurrent all </secondary_problem> with <secondary_problem> cns involvement </secondary_problem> s / p omaya removal due to <primary_problem> vre contamination </primary_problem> & <primary_problem> sdh evacuation </primary_problem>. she is now doing well and <primary_symptom> awake </primary_symptom> s / p extubation, <primary_problem> afebrile </primary_problem> and her wbc count is trending downward. [SEP] <problem> cns vre </problem> : s / p omaya removal, on linezolid for greater cns penetration despite risk of cns penetration. no tee performed. - continue linezolid ( d1 = [ * * 4 - 16 * * ] ), course to be determined by tee ( 4 vs 8 weeks ) gentamicin d / c d per id - f / u culture data and sensitivities - tee deferred at this time. - f / u tigacycline sensitivities for long - term therapy [SEP]
[CLS] the patinet is a [ * * age over 90 * * ] year old female with a history of <secondary_problem> hypertension </secondary_problem> a

In [99]:

# train!!
test_results = trainer.predict(dataset['valid'])    


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: ROW ID, HADM ID, Assessment, Plan Subsection. If ROW ID, HADM ID, Assessment, Plan Subsection are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 6
  Batch size = 4


In [100]:
test_results

PredictionOutput(predictions=array([[ 6.9185734 , -0.92201823, -2.8100662 , -3.7802644 ],
       [ 6.7194004 , -0.7142646 , -2.694345  , -3.913022  ],
       [-2.1814272 ,  5.987942  , -1.461201  , -4.5394278 ],
       [-2.4529886 , -0.26281548,  7.107601  , -3.6556134 ],
       [-2.7313566 ,  6.190248  , -1.3259091 , -4.2794614 ],
       [-1.9983196 , -0.57214653,  6.907893  , -3.3900998 ]],
      dtype=float32), label_ids=array([0, 0, 1, 2, 1, 1]), metrics={'test_loss': 1.2473772764205933, 'test_accuracy': {'accuracy': 0.8333333333333334}, 'test_f1-macro': {'f1': 0.8222222222222223}, 'test_runtime': 7.8954, 'test_samples_per_second': 0.76, 'test_steps_per_second': 0.253})

In [101]:
dataset['valid']['label']

tensor([0, 0, 1, 2, 1, 1])

In [102]:
np.argmax(test_results.predictions, axis=1)

array([0, 0, 1, 2, 1, 2])

In [103]:
dataset['valid'][5]

{'label': tensor(1),
 'input_ids': tensor([    2, 30530,  3546, 14557,    16,  2303, 30531, 30530,  8375,  6189,
          1919,    16,  2826,    12,  5626,  2694, 11788,  2126,    16,  2387,
          3846,  1005,    16,  7217,  6189,  1919,    16, 10122,    13, 30531,
          5439,    17,  2476,    17,  4156,  2323,  1956,  4403,  1927, 30536,
         24078,  8071, 30537,    16, 30536,  5683,  2539, 30537,    16, 30536,
          5235, 30537,  1990,  6434, 19824,  1039,    16, 30536,  3931,  1028,
         30537,    16, 30536,  5579, 30537,  3549,  1956, 30530,  2387,  3846,
          1005, 30531,    16,  2501,  3205, 30530, 24078,  2168,  7633, 30531,
            18,     3,     7, 30528,  6456,  1031, 30529,    30,  3205,  2781,
         16964,  1031,    16,  9040,  1942,  6017,  1956, 20653, 27576,  1036,
            16,  9085,  3004, 29079,  2637,  9759,  2510,  8685,  1007,    16,
          2353,  2572,  5524,  2057,  2647,  4680,    18,  2234,  2258,    43,
          4403,  1